In [ ]:
import autogen

config_list_gpt4o = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ['gpt-4o']
    },
)

In [ ]:
gpt4o_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list_gpt4o,
    "timeout": 120,
}

user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message="A human admin. Interact with a planner to discuss the plan of execution. This plan needs to be approved by this admin.",
    code_execution_config=False
)

planner = autogen.AssistantAgent(
    name='Planner', 
    system_message='Planner. Suggest a plan. Revise the lan based on feedback from a critic agent.\
        THe plan may involve an engineer who can write code and a scientist who doesnt write code.\
        Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a scientist.',
    llm_config = gpt4o_config
)

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=gpt4o_config,
    system_message="""Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
""",
)
scientist = autogen.AssistantAgent(
    name="Scientist",
    llm_config=gpt4o_config,
    system_message="""Scientist. You follow an approved plan. You are able to categorize papers after seeing their abstracts printed.\
        You don't write code. You provided detailed resource reports for the ResearchWriter to write comprehensive research reports.""",
)
executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "paper",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)
critic = autogen.AssistantAgent(
    name="Critic",
    system_message="Critic. Double check, claims, code and report from other agents and provide feedback.\
        Check whether the final research report includes adding verifiable info such as source URL.",
    llm_config=gpt4o_config,
)

research_report_writer = autogen.AssistantAgent(
    name='ResearchWriter',
    system_message='Research Report Writer. Write a research report based on the findings from the papers categorized by the scientist and exchange with critic to improve the quality of the report.\
        The report should include the following sections: Introduction, Literature Review, Methodology, Results, Conclusion, and References. The report should be written in a clear and concise manner.\
        Make sure to include proper citations and references.',
    llm_config=gpt4o_config
)
